In [403]:
import pandas as pd
import numpy as np
import datetime

In [404]:
data = pd.read_csv("hh.csv")
data = data[data.state=="NY"] 
data.head()

,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_adult_covid_suspected_40-49_7_day_sum,previous_day_admission_adult_covid_suspected_50-59_7_day_sum,previous_day_admission_adult_covid_suspected_60-69_7_day_sum,previous_day_admission_adult_covid_suspected_70-79_7_day_sum,previous_day_admission_adult_covid_suspected_80+_7_day_sum,previous_day_admission_adult_covid_suspected_unknown_7_day_sum,previous_day_admission_pediatric_covid_suspected_7_day_sum,previous_day_total_ED_visits_7_day_sum,previous_day_admission_influenza_confirmed_7_day_sum,geocoded_hospital_address
2957,330057,2021/03/05,NY,330057,ST PETER'S HOSPITAL,315 SOUTH MANNING BOULEVARD,ALBANY,12208.0,Short Term,36001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,898.0,0.0,POINT (-73.802382 42.660176)
2958,330013,2021/03/05,NY,330013,ALBANY MEDICAL CENTER HOSPITAL,"43 NEW SCOTLAND AVENUE, MAIL CODE 34",ALBANY,12208.0,Short Term,36001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-999999.0,1419.0,0.0,POINT (-73.77386 42.653254)
2959,330096,2021/03/05,NY,330096,JONES MEMORIAL HOSPITAL,191 NORTH MAIN STREET,WELLSVILLE,14895.0,Short Term,36003.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,171.0,0.0,POINT (-77.949784 42.122878)
2960,331301,2021/03/05,NY,331301,"CUBA MEMORIAL HOSPITAL, INC",140 WEST MAIN STREET,CUBA,14727.0,Critical Access Hospitals,36003.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,POINT (-78.288139 42.213341)
2961,330399,2021/03/05,NY,330399,ST BARNABAS HOSPITAL,4422 THIRD AVENUE,BRONX,10457.0,Short Term,36005.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1077.0,0.0,POINT (-73.892168 40.85277)


In [405]:
cities = ["BRONX", "NEW YORK", "BROOKLYN", "ELMHURST", "FLUSHING", "JAMAICA", "FAR ROCKAWAY", "STATEN ISLAND"]
data = data[data.city.isin(cities)]

In [406]:
data_total = data[["hospital_name", "city", "geocoded_hospital_address","total_icu_beds_7_day_avg","icu_beds_used_7_day_avg", "collection_week", "staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg"]]

# Turn collection week into a datetime object
data_total['collection_week'] = pd.to_datetime(data_total.collection_week)
data_total["pct_used"]=data_total["icu_beds_used_7_day_avg"]/data_total["total_icu_beds_7_day_avg"]


<ipython-input-406-58e3567fc36c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_total['collection_week'] = pd.to_datetime(data_total.collection_week)
<ipython-input-406-58e3567fc36c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_total["pct_used"]=data_total["icu_beds_used_7_day_avg"]/data_total["total_icu_beds_7_day_avg"]


In [407]:
def longsplit(point):
    if not isinstance(point,str):
        return point
    return point.split()[1].strip("(")

def latsplit(point):
    if not isinstance(point,str):
        return point
    return point.split()[2].strip(")")

data_total = data_total.set_index("hospital_name")
data_total.at['MOUNT SINAI HOSPITAL', 'geocoded_hospital_address'] = "POINT (-73.9557924 40.7902121)"
data_total.at['NEW YORK-PRESBYTERIAN/QUEENS', 'geocoded_hospital_address'] = "POINT (-73.9390472 40.7381402)"
data_total.at['NORTH CENTRAL BRONX HOSPITAL', 'geocoded_hospital_address'] = "POINT (-73.8832765 40.8805787)"


data_total["long"] = data_total["geocoded_hospital_address"].apply(longsplit)
data_total["lat"] = data_total["geocoded_hospital_address"].apply(latsplit)

# Dropping rows that do not report ICU bed stats or with negative icu bed usage, which was data ommitted/unreported rows in the original dataset.
data_total = data_total[data_total["icu_beds_used_7_day_avg"]>0]

In [408]:
data_recent = data_total[data_total['collection_week']>pd.to_datetime("03/01/21")]

data_recent.to_csv("data.csv")
print(data_recent.shape)

data_recent.head(50)

(36, 9)


,city,geocoded_hospital_address,total_icu_beds_7_day_avg,icu_beds_used_7_day_avg,collection_week,staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg,pct_used,long,lat
hospital_name,,,,,,,,,
ST BARNABAS HOSPITAL,BRONX,POINT (-73.892168 40.85277),56.0,42.4,2021-03-05,12.1,0.757143,-73.892168,40.85277
JACOBI MEDICAL CENTER,BRONX,POINT (-73.84721 40.857455),89.0,69.3,2021-03-05,14.7,0.778652,-73.84721,40.857455
LINCOLN MEDICAL & MENTAL HEALTH CENTER,BRONX,POINT (-73.924072 40.817681),79.1,48.1,2021-03-05,13.4,0.608091,-73.924072,40.817681
BRONXCARE HOSPITAL CENTER,BRONX,POINT (-73.903756 40.83175),77.0,45.9,2021-03-05,17.7,0.596104,-73.903756,40.83175
NORTH CENTRAL BRONX HOSPITAL,BRONX,POINT (-73.8832765 40.8805787),65.0,50.3,2021-03-05,32.1,0.773846,-73.8832765,40.8805787
MONTEFIORE MEDICAL CENTER,BRONX,POINT (-73.879975 40.879777),206.0,180.6,2021-03-05,51.9,0.876699,-73.879975,40.879777
CONEY ISLAND HOSPITAL CENTER,BROOKLYN,POINT (-73.966285 40.58494),56.3,45.1,2021-03-05,14.3,0.801066,-73.966285,40.58494
KINGS COUNTY HOSPITAL CENTER,BROOKLYN,POINT (-73.944039 40.655773),54.0,54.3,2021-03-05,6.7,1.005556,-73.944039,40.655773
BROOKDALE HOSPITAL MEDICAL CENTER,BROOKLYN,POINT (-73.91275 40.6545),51.1,43.3,2021-03-05,10.9,0.847358,-73.91275,40.6545


In [409]:
last_90 = data_total[data_total['collection_week']>pd.to_datetime("12/01/20")]
last_90 = last_90.reset_index()
last_90 = last_90[["hospital_name", "lat", "long", "collection_week", "pct_used"]]

pivoted = last_90.pivot(index=["hospital_name", "lat", "long"], columns="collection_week")
pivoted = pivoted.reset_index()

# Fix column headers after the pivot
level0 = [a for a in pivoted.columns.get_level_values(0).values[0:3]]
level1 = [a.strftime('%Y-%m-%d') for a in pivoted.columns.get_level_values(1).values[3:]]
pivoted.columns = level0 + level1

pivoted.to_csv("pivoted.csv")
print(pivoted.shape)
pivoted.head(60)

(36, 17)


,hospital_name,lat,long,2020-12-04,2020-12-11,2020-12-18,2020-12-25,2021-01-01,2021-01-08,2021-01-15,2021-01-22,2021-01-29,2021-02-05,2021-02-12,2021-02-19,2021-02-26,2021-03-05
0,BELLEVUE HOSPITAL CENTER,40.740079,-73.97625,0.997379,0.982565,0.850000,0.791045,0.738679,0.807547,0.843396,0.850000,0.879541,0.866981,0.880189,0.831250,0.832136,0.828190
1,BRONXCARE HOSPITAL CENTER,40.83175,-73.903756,0.575325,0.524675,0.559740,0.588312,0.605195,0.619481,0.650649,0.645455,0.687013,0.650649,0.592208,0.584416,0.559740,0.596104
2,BROOKDALE HOSPITAL MEDICAL CENTER,40.6545,-73.91275,0.932563,0.936735,0.832661,0.834677,0.805726,0.855159,0.889546,0.901186,0.969231,0.866419,0.914657,0.933723,0.903226,0.847358
3,BROOKLYN HOSPITAL CENTER - DOWNTOWN CAMPUS,40.689785,-73.9774,0.495556,0.596659,0.691667,0.541667,0.579167,0.704167,0.462500,0.683333,0.754167,0.666667,0.612500,0.683333,0.712500,0.766667
4,CONEY ISLAND HOSPITAL CENTER,40.58494,-73.966285,0.585570,0.533333,0.624374,0.714801,0.795259,0.825103,0.903922,0.898662,0.874101,0.889876,0.834559,0.760984,0.832759,0.801066
5,ELMHURST HOSPITAL CENTER,40.74434,-73.88598,0.179213,0.198544,0.155905,0.212508,0.446988,0.504819,0.512338,0.727273,0.698027,0.519277,0.531325,0.514458,0.534940,0.525301
6,FLUSHING HOSPITAL MEDICAL CENTER,40.755636,-73.815645,0.991667,0.905556,0.747222,0.611111,0.711111,0.702778,0.697222,0.766667,0.841667,0.777778,0.825000,0.730556,0.786111,0.663889
7,HARLEM HOSPITAL CENTER,40.814482,-73.940495,0.825000,0.846429,0.942857,1.010830,0.910299,0.762346,0.735202,0.814189,0.712991,0.820000,0.754579,0.648352,0.766871,0.786477
8,INTERFAITH MEDICAL CENTER,40.677866,-73.937154,0.969231,0.915385,0.669231,0.869231,0.575419,0.930769,0.601852,0.739130,0.843478,0.808696,0.743478,0.647826,0.691304,0.808696
9,JACOBI MEDICAL CENTER,40.857455,-73.84721,0.319048,0.309524,0.334762,0.430443,0.761856,0.700000,0.748454,0.839145,0.803456,0.794183,0.774578,0.763485,0.807865,0.778652
